In [12]:
messung ="1"

In [33]:
import yaml

import os
import sys

import matplotlib.pyplot as plt

notebook_path = os.path.abspath('')
project_root = os.path.dirname(notebook_path)
sys.path.append(project_root)

from modules.filters import *

from helpers.read_osc_to_df import *

In [93]:
# Import Config dict
config_dict=yaml.load(open("../config/coincidence_config.yaml", "r"),yaml.SafeLoader)["find_peaks"]
config_dict

# Config from find peaks
if config_dict is None:
    raise ValueError("ERROR! Wrong configuration passed (in lifetime_modules: calculate_decay_time)!!")

# Load configuration
sample_time_ns = config_dict["sample_time_ns"]
number_of_samples = config_dict["number_of_samples"]
analogue_offset = config_dict["analogue_offset"]*1000
pre_trigger_samples = config_dict["pre_trigger_samples"]
trigger_channel = 'ch'+config_dict["trigger_channel"]
coincidence_window = config_dict["coincidence_window"]


peak_config = config_dict["peak_config"]


list_of_channels = config_dict["list_of_channels"]
clipping_level = config_dict["clipping_level"]


if trigger_channel not in list_of_channels:
    raise ValueError(f'{trigger_channel} not in list of channels: {list_of_channels}') 
trigger_position_tolerance = config_dict["trigger_position_tolerance"]

signal_channels = [channel for channel in list_of_channels if channel != trigger_channel]


# nur weil net aus Sink List importierbar
pulse_par_dtype = np.dtype([('ch1_position', 'i4'), ('ch1_height', 'f4'), ('ch2_position', 'i4'), ('ch2_height', 'f4')])

# glaub neu
empty_peak_data=np.zeros( (1,), dtype=pulse_par_dtype)

In [94]:
#load Data
data,number_of_triggers=import_data(project_root+ "/measurements/measurement_3_" + messung +"min.npy",number_of_samples, list_of_channels)

Data read successfully, recorded triggers = 2752


In [114]:


# define tag_pulses
def tag_pulses(input_data):   
    """find all valid pulses 

    This function to be called by instance of class mimoCoRB.rbTransfer

        Args:  input data as structured ndarray

        Returns: list of parameterized pulses
        
        
        Currently only works with two channels. Every part which requires two channels ist marked with ### two
    """
    # Find all the peaks and store them in a dictionary
    
    
    #this is ugly :(
    #peaks, peaks_prop = tag_peaks(input_data, peak_minimal_prominence, peak_minimal_distance, peak_minimal_width, peak_gradient_bound["upper"],peak_gradient_bound["lower"])
    peaks, peaks_prop = tag_peaks(input_data, peak_config)

    peak_data=empty_peak_data.copy()
    
    # Only consider events with one peak in the trigger_channel
    if len(peaks[trigger_channel])!=1:
        return None
    trigger_peak=peaks[trigger_channel][0]
    for signal_channel in signal_channels:
        if len(peaks[signal_channel])!=1: ### two
            return None ### two
        Delta_T=coincidence_window[signal_channel]['offest_from_trigger']/sample_time_ns
        w=coincidence_window[signal_channel]['width_of_window']/sample_time_ns
        for peak in peaks[signal_channel]:
            # filter out clipped peaks
            if input_data[signal_channel][peak]>=clipping_level:
                print('clipped peak')
                return None
            # check for coincidences
            if -w/2<=peak-Delta_T-trigger_peak<=w/2:
                found_coincidence=True
                # explanation: t+Delta_T-w/2<=peak<=t+Delta_T+w/2
                """ab hier erstmal nur für einen trigger und einen signal channel"""
                peak_data[0][signal_channel+'_position'] = peak
                peak_data[0][trigger_channel+'_position'] = trigger_peak
                peak_data[0][signal_channel+'_height'] = peaks_prop[signal_channel]['relative_height'][0] # das muss später weg wenn man mehr als ein peak betrachtet
                peak_data[0][trigger_channel+'_height'] = peaks_prop[trigger_channel]['relative_height'][0]
    
    if peak_data!=empty_peak_data:
        print(peak_data)
        return peak_data
    else: 
        return None

In [115]:
for d in data:
    t=tag_pulses(d)
    if t!=None:
        d_out=d
plt.plot(d_out['ch1'])
plt.plot(d_out['ch2'])


[(127, 4110., 130, 873.)]


TypeError: Cannot compare structured or void to non-void arrays.

In [120]:
empty_peak_data

array([(0, 0., 0, 0.)],
      dtype=[('ch1_position', '<i4'), ('ch1_height', '<f4'), ('ch2_position', '<i4'), ('ch2_height', '<f4')])